# 4. Introducing the Grid Network


In the previous tutorials we explicitly connected to the workers in order to interact with them, this can be useful for use cases where we want to have control over where the data is stored (which node?) and who performs each role on the application.

Another possible way to see Grid is that workers that are connected consist of a Grid Network.


## What is the Grid Network?

<p align="center">
<img height="600px" width="600px" src="https://github.com/OpenMined/rfcs/blob/master/20190821-grid-platform/DHT-grid.png?raw=true">
</p>


This network can be seen by a user as a single interface: the Grid Gateway. The Gateway works like a special DNS component but it will route nodes by queries instead of domain names.

It is important to emphasize: the Grid Gateway will not be able to perform any computation process on the nodes. It can not concentrate or centralize any data or model. It works as a brigde between a user that is outside of the grid network to get data / models that live inside the network.


<p align="center">
<img height="600px" width="600px" src="https://github.com/OpenMined/rfcs/blob/master/20190821-grid-platform/partially_grid.png?raw=true">
</p>

With this fully distributed and descentralized architecure we can provide fault tolerance in a very transparent way.


## Using the Grid Network to provide fault tolerance


In this notebook we'll use a grid network to host the CNN model trained on Part 3.1. This model is used for classifying an image as melanoma or benign keratosis as an use case.

In [ ]:
import torch as th
import grid as gr
import grid.syft as sy
import skin_cancer_model_utils as scmu

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

hook = sy.TorchHook(th)

## Connect to the Grid Network

We don't need to connect to each worker explictly we just connect to a gateway.

In [ ]:
# Checkout the example folders to learn how to setup a GridNetwork on Heroku!
gateway = gr.GridNetwork("https://opengrid.herokuapp.com/")

## Prepare data and model

We prepare the data and the model the same way we do for the grid node.

In [ ]:
data, target = scmu.get_data_sample()

In [ ]:
model = scmu.make_model(is_plan=True, model_id="skin-cancer-grid-model")
model.load_state_dict(th.load("skin-cancer-model"))

In [ ]:
model.eval()

In [ ]:
model.build(data)

### Serve the model 

The gateway interface is the same of a grid node. The difference is that the gateway does not hold any object (data, model), it asks the node to perform the operations and hold the objects.

In [ ]:
gateway.serve_model(model, "skin-cancer-grid-model", allow_remote_inference=True, allow_download=False)

### Querying a model

In [ ]:
gateway.run_remote_inference(model_id="skin-cancer-grid-model", dataset=data)

### Disconnect from the network

In [ ]:
gateway.disconnect_nodes()